In [1]:
#=========================================================================================================
#문장쌍 STS dataset 이 없는 경우 예시
#
# - 1단계: 기존 korsts, kluests 등 sts dataset을 가지고, Cross-Encoder로 BERT 훈련 시킴
# - 2단계: 1단계에서 훈련시킨 BERT를 Cross-Encoder로 label이 없는 sts dataset에 대해 label을 지정함
# - 3단계: label이 지정된 sts dataset에 대해 Bi-Encoder 훈련 시킴
#=========================================================================================================

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

import math
from sentence_transformers import models, losses
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import sys
import os
import gzip
import csv

sys.path.append('../../')
from myutils import seed_everything, GPU_info, mlogging

logger = mlogging(loggername="s-bert", logfilename="../../log/s-bert")
device = GPU_info()
seed_everything(111)

logfilepath:../../log/bwdataset_2022-04-25.log
logfilepath:../../log/qnadataset_2022-04-25.log
logfilepath:../../log/s-bert_2022-04-25.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
# 모델들 정의
max_seq_len = 128
train_batch_size = 32
num_epochs = 10
top_k = 3             # 훈련데이터에서 몇개의 유사도 문장을 뽑아낼지 정하는 값

#======================================================================================================
# cross-encoder 모델 정의
ce_model_path = "../../../model/bert/bmc-fpt-bong_corpus_mecab-0424/batch:32-ep:4-lr:0.000030000-4m25d-8:27/"
ce_model = CrossEncoder(ce_model_path, num_labels=1)
#======================================================================================================
print(ce_model)


Some weights of the model checkpoint at ../../../model/bert/bmc-fpt-bong_corpus_mecab-0424/batch:32-ep:4-lr:0.000030000-4m25d-8:27/ were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [5]:
#  1단계: 기존 korsts, kluests 등 sts dataset을 가지고, Cross-Encoder로 BERT 훈련 시킴
# => 여기서는 kluests를 가지고 훈련시킴
import json

train_file = '../../../korpora/klue-sts/klue-sts-v1.1_train.json'
eval_file = '../../../korpora/klue-sts/klue-sts-v1.1_dev.json'
count = 0

# 훈련 골드 데이터 불러옴
train_gold_samples = []
with open(train_file, "rt", encoding="utf-8") as f:
    datas = json.load(f)
    for data in datas:
        text_a = data["sentence1"]
        text_b = data["sentence2"]
        score = data["labels"]["label"]
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함

        if count < 5:
            print(f"{text_a}, {text_b}, {score}")

        train_gold_samples.append(InputExample(texts= [text_a,text_b], label=score))
        count += 1
      
# 평가 데이터 불러옴
count = 0
dev_samples = []
with open(eval_file, 'rt', encoding='utf-8') as f:
    datas = json.load(f)
    for data in datas:
        text_a = data["sentence1"]
        text_b = data["sentence2"]
        score = data["labels"]["label"]
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함

        if count < 5:
            print(f"{text_a}, {text_b}, {score}")

        dev_samples.append(InputExample(texts= [text_a,text_b], label=score))
        count += 1
        
# 훈련 골드 데이터로더, 평가 데이터 로더 생성
train_dataloader = DataLoader(train_gold_samples, shuffle=True, batch_size=train_batch_size)
evaluator = CECorrelationEvaluator.from_input_examples(dev_samples, name='sts-dev')

print(train_gold_samples[0:2])
print(dev_samples[0:2])

숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다., 숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다., 0.74
위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다., 시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다., 0.0
회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘., 사람들이 주로 네이버 메일을 쓰는 이유를 알려줘, 0.06
긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업성공패키지, 청년구직활동지원금, 긴급복지지원제도 지원금과는 중복 수급이 불가능하다., 고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다., 0.12
호스트의 답장이 늦으나, 개선될 것으로 보입니다., 호스트 응답이 늦었지만 개선될 것으로 보입니다., 0.9400000000000001
무엇보다도 호스트분들이 너무 친절하셨습니다., 무엇보다도, 호스트들은 매우 친절했습니다., 0.9800000000000001
주요 관광지 모두 걸어서 이동가능합니다., 위치는 피렌체 중심가까지 걸어서 이동 가능합니다., 0.27999999999999997
학생들의 균형 있는 영어능력을 향상시킬 수 있는 학교 수업을 유도하기 위해 2018학년도 수능부터 도입된 영어 영역 절대평가는 올해도 유지한다., 영어 영역의 경우 학생들이 한글 해석본을 암기하는 문제를 해소하기 위해 2016학년도부터 적용했던 EBS 연계 방식을 올해도 유지한다., 0.26
다만, 도로와 인접해서 거리의 소음이 들려요., 하지만, 길과 가깝기 때문에 거리의 소음을 들을 수 있습니다., 0.74
형이 다시 캐나다 들어가야 하니 가족모임 일정은 바꾸지 마세요., 가족 모임 일정은 바꾸지 말도록 하십시오., 0.5
[<sentence_transformers.readers.InputExample.InputExample object at 0x7f1e4817b7f0>, <sentence

In [6]:
# 1단계: 적은 STS dataset(gold sts dataset) 에 대해 Cross-Encoder로 BERT 훈련 시킴

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logger.info("Warmup-steps: {}".format(warmup_steps))

ce_odel_save_path = 'output/crossencoder-sts-train-'+datetime.now().strftime("%Y-%m-%d-%H:%M")

# Train the model
ce_model.fit(train_dataloader=train_dataloader,
          evaluator=evaluator,
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          output_path=ce_odel_save_path)

2022-04-25 14:19:13,002 - s-bert - INFO - Warmup-steps: 365


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

In [8]:
# 2단계: 1단계에서 훈련시킨 BERT를 Cross-Encoder로 label이 없는 sts dataset에 대해 label을 지정함
# => 여기서는 sts dataset도 없고, 그냥 문장 말뭉치만 있는 경우를 가정함.

#======================================================================================================
# 훈련 데이터들의 유사도 문장들를 구할 bi-encoder 모델 정의
# => paraphrase-multilingual-mpnet-base-v2 를 이용
bi_encoder_path = "../../../model/sbert/teacher/paraphrase-multilingual-mpnet-base-v2/"

word_embedding_encoder = models.Transformer(bi_encoder_path, max_seq_length=max_seq_len)

pooling_encoder = models.Pooling(word_embedding_encoder.get_word_embedding_dimension(),  #모델이 dimension(768)
                               pooling_mode_mean_tokens=True,  # 워드 임베딩 평균을 이용
                               pooling_mode_cls_token=False,   # cls 를 이용
                               pooling_mode_max_tokens=False)  # 워드 임베딩 값중 max 값을 이용

bi_encoder = SentenceTransformer(modules=[word_embedding_encoder, pooling_encoder])
#======================================================================================================

# 문장들을 불러옴.
corpus_path = '../../../korpora/kowiki_20190620/wiki_20190620_small.txt'
sentneces = []
with open(corpus_path, encoding="utf-8") as f:
    sentences = [line for line in tqdm(f.read().splitlines()) if (len(line) > 0 and not line.isspace())]
    
# 문장들의 embedding을 구함.
embeddings = bi_encoder.encode(sentences, batch_size=train_batch_size, convert_to_tensor=True)

# 각 문장들에서 유사도가 높은 문장들을 쌍으로 묶음
duplicates = set()
silver_train_data = []

for idx in tqdm(range(len(sentences)), unit="docs"):
    sentence_embedding = embeddings[idx]
    cos_scores = util.pytorch_cos_sim(sentence_embedding, embeddings)[0]
    cos_scores = cos_scores.cpu()
    top_results = torch.topk(cos_scores, k=top_k+1)
    
    for score, iid in zip(top_results[0], top_results[1]):
        if iid != idx and (iid, idx) not in duplicates:
            silver_train_data.append((sentences[idx], sentences[iid]))
            duplicates.add((idx, iid))

# 신규 데이터 말뭉치에 대해 score를 추가함
cs_new_model = CrossEncoder(ce_odel_save_path)
silver_scores = cs_new_model.predict(silver_train_data)

print(len(silver_train_data))
print(silver_train_data[0:5])
print(silver_scores[0:5])

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/9818 [00:00<?, ?docs/s]

29504
[('제임스 얼 "지미" 카터 주니어는 민주당 출신 미국 39번째 대통령 이다.', '46세의 나이로 대통령이 된 그는 역대 미국 대통령 중에서 세 번째로 젊은 대통령이었다.'), ('제임스 얼 "지미" 카터 주니어는 민주당 출신 미국 39번째 대통령 이다.', '클린턴은 프랭클린 루스벨트 이후로 두번의 임기를 모두 채운 첫 번째 민주당 대통령이었다.'), ('제임스 얼 "지미" 카터 주니어는 민주당 출신 미국 39번째 대통령 이다.', '1993년 민주당 최연소 최고위원이 되었다.'), ('지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.', '그녀는 미국의 남부 조지아주 애틀랜타에서 태어났다.'), ('지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.', '제임스 얼 "지미" 카터 주니어는 민주당 출신 미국 39번째 대통령 이다.')]
[0.26152834 0.30554742 0.04845052 0.49450356 0.35710645]


In [9]:
# silver_data 를 .tsv 파일로 저장 해둠.
silver_data_file = 'output/silver_data-'+datetime.now().strftime("%Y-%m-%d-%H:%M")+'.tsv'
with open(silver_data_file, 'w', encoding='utf-8') as f:
    for data, score in tqdm(zip(silver_train_data, silver_scores)):
        f.write(data[0]+'\t')
        f.write(data[1]+'\t')
        f.write(str(score) + '\n')    

0it [00:00, ?it/s]

In [11]:
# 3단계: gold sts dataset + silver sts dataset 을 훈련 데이터로 하여 Bi-Encoder 훈련 시킴

#======================================================================================================
# bi-encoder 모델 정의

# word_embedding 모델은 앞에서 할 cross-encoder 모델 경로로 지정함 
word_embedding_model = models.Transformer(ce_model_path, max_seq_length=max_seq_len)

pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),  #모델이 dimension(768)
                               pooling_mode_mean_tokens=True,  # 워드 임베딩 평균을 이용
                               pooling_mode_cls_token=False,   # cls 를 이용
                               pooling_mode_max_tokens=False)  # 워드 임베딩 값중 max 값을 이용

bi_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
#======================================================================================================

train_silver_samples = list(InputExample(texts=[data[0], data[1]], label=score) for \
    data, score in zip(silver_train_data, silver_scores))

train_dataset = SentencesDataset(train_gold_samples + train_silver_samples, bi_model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=bi_model)
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')

# Configure the training.
warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up
#logging.info("Warmup-steps: {}".format(warmup_steps))

model_save_path = 'output/sbert-sts-train-'+datetime.now().strftime("%Y-%m-%d_%H-%M")

# Train the bi-encoder model
bi_model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path
          )


Some weights of the model checkpoint at ../../../model/bert/bmc-fpt-bong_corpus_mecab-0424/batch:32-ep:4-lr:0.000030000-4m25d-8:27/ were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ../../../model/bert/bmc-fpt-bong_

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1287 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1287 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1287 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/1287 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1287 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1287 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1287 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1287 [00:00<?, ?it/s]